### Web-Scraping-Project-4

In [1]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyperclip

In [8]:
#sendng request to the page
url = "https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos"
page = requests.get(url)
soup = BeautifulSoup(page.content)
#getting the the table
table = soup.find("table", ["sortable", "wikitable", "jquery-tablesorter"])
#fin all table rows
trs = table.find_all("tr")
#initializing lists
ranks = []
names = []
artists = []
upload_dates = []
views = []
#looping through trs excluding heading and last row
for tr in trs[1:-1]:
    tds = tr.find_all("td")
    ranks.append(tds[0].text)
    names.append(tds[1].text.split("[")[0].strip("\""))
    artists.append(tds[2].text)
    upload_dates.append(tds[4].text)
    views.append(tds[3].text)
    
#creating dataframe
youtube_views = pd.DataFrame({"Rank":ranks, "Name":names, "Artist":artists, "Upload Date":upload_dates, 
                              "Views(in Billions)":views})
youtube_views

,Rank,Name,Artist,Upload Date,Views(in Billions)
0,1.,Baby Shark Dance,Pinkfong Kids' Songs & Stories,"June 17, 2016",8.71
1,2.,Despacito,Luis Fonsi,"January 12, 2017",7.38
2,3.,Johny Johny Yes Papa,LooLoo Kids,"October 8, 2016",5.41
3,4.,Shape of You,Ed Sheeran,"January 30, 2017",5.34
4,5.,See You Again,Wiz Khalifa,"April 6, 2015",5.13
5,6.,Masha and the Bear – Recipe for Disaster,Get Movies,"January 31, 2012",4.44
6,7.,Uptown Funk,Mark Ronson,"November 19, 2014",4.19
7,8.,Learning Colors – Colorful Eggs on a Farm,Miroshka TV,"February 27, 2018",4.12
8,9.,Bath Song,Cocomelon – Nursery Rhymes,"May 2, 2018",4.10
9,10.,Gangnam Style,Psy,"July 15, 2012",4.08


In [2]:
#initialize web driver
driver = webdriver.Firefox(executable_path = r"C:/Users/DEVASREE/Downloads/geckodriver.exe")
url = "https://www.bcci.tv/"
driver.get(url)
time.sleep(3)

In [22]:
#find the internatin menu and extract url to fixture page
international = driver.find_element_by_xpath("//div[@class='navigation__drop-down drop-down drop-down--reveal-on-hover']")
fixture_url = international.find_element_by_tag_name("a").get_attribute("href")
driver.get(fixture_url)
time.sleep(5)
#finding individual match div elements
divs = driver.find_elements_by_class_name("event-list__item")
#initializing lists
titles = []
series = []
places = []
dates = []
times = []
#looping through each div to get information
for j,div in enumerate(divs):
    try:
        #addming info to the lists
        titles.append(div.find_element_by_class_name("fixture__name").text)
        ser = div.find_element_by_class_name("fixture__format-strip")
        #getting the info of match(test or odi) + series name
        typ =  str(ser.find_elements_by_tag_name("span")[0].get_attribute("innerHTML")) + \
        ":" + str(ser.find_elements_by_tag_name("span")[1].get_attribute("innerHTML"))
        series.append(typ)
        places.append(div.find_element_by_class_name("fixture__additional-info").\
                      find_element_by_tag_name("span").get_attribute("innerHTML"))
        #finding the individual div for date and extracting seperately
        date_div = div.find_elements_by_xpath("//div[@class='fixture__datetime desktop-only']")
        day = date_div[j].find_element_by_class_name("fixture__day").text
        #uniting the date information without \n
        fix_full_date = day + ", " + date_div[j].find_element_by_class_name("fixture__date").text\
        + "-" +date_div[j].find_element_by_class_name("fixture__month").text
        dates.append(fix_full_date)
        #extracting time information from date div
        tim = date_div[j].find_element_by_class_name("fixture__time")
        times.append(tim.text)
    except NoSuchElementException as e:
        print(str(e))
    except StaleElementReferenceException as e:
        print(str(e))
#creating dataframe
fixtures = pd.DataFrame({"Title":titles, "Series":series, "Place":places, "Date":dates, "Time":times})
fixtures

,Title,Series,Place,Date,Time
0,Final,Test:ICC World Test Championship,"The Ageas Bowl, Southampton","Friday, 18-JUNE",15:00 IST
1,1st Test,Test:England v India 2021,"Trent Bridge, Nottingham","Wednesday, 04-AUGUST",15:30 IST
2,2nd Test,Test:England v India 2021,"Lord's, London","Thursday, 12-AUGUST",15:30 IST
3,3rd Test,Test:England v India 2021,"Headingley, Leeds","Wednesday, 25-AUGUST",15:30 IST
4,4th Test,Test:England v India 2021,"The Oval, London","Thursday, 02-SEPTEMBER",15:30 IST
5,5th Test,Test:England v India 2021,"Old Trafford, Manchester","Friday, 10-SEPTEMBER",15:30 IST


In [45]:
#starting web driver
driver = webdriver.Firefox(executable_path = r"C:/Users/DEVASREE/Downloads/geckodriver.exe")
url = "https://www.guru99.com"
driver.get(url)
time.sleep(3)

In [46]:
#finding all the boxes with contents
list_divs = driver.find_elements_by_class_name("featured-box")
#iterating through this div to find the selenium section
br = False
for j,list_div in enumerate(list_divs):
    list1 = list_div.find_elements_by_class_name("menu")\
        [j].find_elements_by_tag_name("a")
    #iterating through menu items to find selenium section
    for a in list1:
        if "selenium" in a.text.lower():
            #getting url for selenium
            href = a.get_attribute("href")
            #redirecting to the url
            driver.get(href)
            br = True
            break
    if br:
        break
        

In [51]:
#find all rows which tutorial link  is given
rows = driver.find_elements_by_tag_name("tr")
#looping through rows
for row in rows:
    #check the title contains "Exception handling" or not
    if "exception handling" in row.text.lower():
        #getting url and redirects
        driver.get(row.find_element_by_tag_name("a").get_attribute("href"))
        break

In [54]:
#finding the table 
table = driver.find_element_by_xpath("//table[@class='table table-striped']")
#getting every row of the table
rows = table.find_elements_by_tag_name("tr")
#initializinf empty lists
exceptions = []
descriptions = []
#looping through rows
for row in rows[1:]:
    exceptions.append(row.find_elements_by_tag_name("td")[0].text)
    descriptions.append(row.find_elements_by_tag_name("td")[1].text)
#creating dataframe
df = pd.DataFrame({"Name of Exception":exceptions, "Description":descriptions})
df

,Name of Exception,Description
0,ElementNotVisibleException,This type of Selenium exception occurs when an...
1,ElementNotSelectableException,This Selenium exception occurs when an element...
2,NoSuchElementException,This Exception occurs if an element could not ...
3,NoSuchFrameException,This Exception occurs if the frame target to b...
4,NoAlertPresentException,This Exception occurs when you switch to no pr...
5,NoSuchWindowException,This Exception occurs if the window target to ...
6,StaleElementReferenceException,This Selenium exception occurs happens when th...
7,SessionNotFoundException,The WebDriver is acting after you quit the bro...
8,TimeoutException,Thrown when there is not enough time for a com...
9,WebDriverException,This Exception takes place when the WebDriver ...


<h2>Note</h2>
<h3>GSDP(18-19) has changed to GSDP(19-20)</h3>
<h3>GSDP(17-18) has changed to GSDP(18-19)</h3>
<h3>Share(2017) has changed to Share(18-19)</h3>

In [2]:
#starting web driver
driver = webdriver.Firefox(executable_path = r"C:/Users/DEVASREE/Downloads/geckodriver.exe")
url = "http://statisticstimes.com"
driver.get(url)
time.sleep(3)

In [25]:
#geting the econmy section
economy_dropdown = driver.find_elements_by_class_name("dropdown")[1]
#getting the stat url
st_url = economy_dropdown.find_elements_by_tag_name("a")[2].get_attribute("href")

#redirecting to statistcs url
driver.get(st_url)

#getting all urls in the page
urls = driver.find_elements_by_class_name("ec")
gdp_url = ""
for ur in urls:
    if "gdp of indian states" in ur.text.lower():
        gdp_url = ur.get_attribute("href")
        break

#redirect to state GDP page
driver.get(gdp_url)
time.sleep(5)


In [33]:
#finding the info table using id
table = driver.find_element_by_id("table_id_wrapper")
tbody = table.find_element_by_tag_name("tbody")
trs = tbody.find_elements_by_tag_name("tr")
#initializing info. lists
ranks = []
states = []
gsdps_20 = []
gsdps_19 = []
shares = []
gdps = []
for tr in trs:
    tds = tr.find_elements_by_tag_name("td")
    ranks.append(tds[0].text)
    states.append(tds[1].text)
    gsdps_20.append(tds[2].text)
    gsdps_19.append(tds[3].text)
    shares.append(tds[4].text)
    gdps.append(tds[5].text)

#creating Dataframe
state_gdp = pd.DataFrame({"Rank":ranks, "State":states, "GSDP(19-20)":gsdps_20,
                         "GSDP(18-19)":gsdps_19, "Share":shares, "GDP":gdps})
print(state_gdp)

   Rank                      State GSDP(19-20) GSDP(18-19)   Share      GDP
0     1                Maharashtra           -   2,632,792  13.94%  399.921
1     2                 Tamil Nadu   1,845,853   1,630,208   8.63%  247.629
2     3              Uttar Pradesh   1,687,818   1,584,764   8.39%  240.726
3     4                    Gujarat           -   1,502,899   7.96%  228.290
4     5                  Karnataka   1,631,977   1,493,127   7.91%  226.806
5     6                West Bengal   1,253,832   1,089,898   5.77%  165.556
6     7                  Rajasthan   1,020,989     942,586   4.99%  143.179
7     8             Andhra Pradesh     972,782     862,957   4.57%  131.083
8     9                  Telangana     969,604     861,031   4.56%  130.791
9    10             Madhya Pradesh     906,672     809,592   4.29%  122.977
10   11                     Kerala           -     781,653   4.14%  118.733
11   12                      Delhi     856,112     774,870   4.10%  117.703
12   13     

In [141]:
#starting web driver
driver = webdriver.Chrome(executable_path = r"C:/Users/DEVASREE/Downloads/chromedriver.exe")
url = "https://github.com"
driver.get(url)
driver.maximize_window()
actions = ActionChains(driver)
time.sleep(3)

In [142]:
#getting the explore menu and click
hover_menu = driver.find_elements_by_class_name("HeaderMenu-summary")[1]
hover_menu.click()
#getting and clicking the sub menu "trending"
click_menu = driver.find_element_by_xpath("//a[@href='/trending']")
click_menu.click()

In [143]:
#getting the list of repeating base elements
articles = driver.find_elements_by_tag_name("article")
#initializing lists
titles = []
descriptions = []
c_counts = []
languages = []

#looping through articles to get info
for article in articles:
    titles.append(article.find_element_by_tag_name("h1").text)
    try:
        descriptions.append(article.find_element_by_tag_name("p").text)
    except:
        descriptions.append("--")
    try:
        built_spans = article.find_elements_by_tag_name("span")
        for cr in built_spans:
            if "built by" in cr.text.lower():
                creators = cr.find_elements_by_tag_name("a")
                c_counts.append(len(creators))
                break
    except:
        c_counts.append("0")
    try:
        lang = article.find_element_by_xpath(".//span[@itemprop='programmingLanguage']").text
        languages.append(lang)
    except:
        languages.append("--")
    
# creating a dataframe
repos = pd.DataFrame({"Title":titles, "Description":descriptions, "Creator Counts":c_counts,
                     "Language": languages})
repos

,Title,Description,Creator Counts,Language
0,madMAx43v3r / chia-plotter,--,5,C
1,peng-zhihui / XUAN-Bike,--,1,--
2,kingoflolz / mesh-transformer-jax,Model parallel transformers in JAX and Haiku,5,Jupyter Notebook
3,jwasham / coding-interview-university,A complete computer science study plan to beco...,5,--
4,microsoft / playwright,"Node.js library to automate Chromium, Firefox ...",5,TypeScript
5,zzanehip / The-OldOS-Project,Recreating a fully functional version of iOS 4...,2,Swift
6,TuSimple / naive-ui,A Vue 3 Component Library. Fairly Complete. Cu...,5,TypeScript
7,drawrowfly / tiktok-scraper,"TikTok Scraper. Download video posts, collect ...",5,TypeScript
8,reactwg / react-18,Workgroup for React 18 release.,3,--
9,chinnkarahoi / jd_scripts,lxk0301/jd_scripts备份。不要star。不要fork。谢谢配合。,5,JavaScript


In [164]:
#starting web driver
driver = webdriver.Chrome(executable_path = r"C:/Users/DEVASREE/Downloads/chromedriver.exe")
url = "https://www.billboard.com/"
driver.get(url)
time.sleep(3)

In [165]:
#get all the links 
links = driver.find_elements_by_class_name("header__main-link__wrapper")
#getting the "charts" link
for link in links:
    if "chart" in link.text.lower():
        link.click()
        break
time.sleep(5)

#scroll down to "the hot 100" secion
value = 0
for i in range(0,3):
    driver.execute_script("scrollBy(" + str(value) + ", +1000);")
    value = value + 1000
    time.sleep(2)
time.sleep(3)
#get links 
hot_links = driver.find_elements_by_xpath("//a[@class='chart-panel__link']")
#clicking the "the hot 100" link
for hot_link in  hot_links:
    if 'the hot 100' in hot_link.text.lower():
        hot_link.click()
        break
time.sleep(5)


In [54]:
#find all lis which consist of music records
lis = driver.find_elements_by_class_name("chart-list__element")
#initializing lists
songs = []
artists = []
lw_rank = []
p_rank = []
weeks = []
#looping through individual lis
for li in lis:
    songs.append(li.find_element_by_class_name("chart-element__information__song").text)
    artists.append(li.find_element_by_class_name("chart-element__information__artist").text)
    metas = li.find_elements_by_class_name("chart-element__metas")[1]
    lw_rank.append(metas.find_element_by_class_name("text--last").get_attribute("innerHTML"))
    p_rank.append(metas.find_element_by_class_name("text--peak").get_attribute("innerHTML"))
    weeks.append(metas.find_element_by_class_name("text--week").get_attribute("innerHTML"))

    #creating Dataframe
songs = pd.DataFrame({"Song":songs, "Artist":artists, "Last Week Rank":lw_rank, 
                     "Peak Rank":p_rank, "Weeks on Board":weeks})
songs

,Song,Artist,Last Week Rank,Peak Rank,Weeks on Board
0,Butter,BTS,1,1,2
1,Good 4 U,Olivia Rodrigo,2,1,3
2,Levitating,Dua Lipa Featuring DaBaby,4,2,35
3,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),5,1,13
4,Save Your Tears,The Weeknd & Ariana Grande,7,1,25
...,...,...,...,...,...
95,All I Know So Far,P!nk,87,74,3
96,Hold On,Justin Bieber,86,20,13
97,Wasted On You,Morgan Wallen,98,9,19
98,Outside,MO3 X OG Bobby Billions,-,99,1


In [2]:
#starting web driver
driver = webdriver.Chrome(executable_path = r"C:/Users/DEVASREE/Downloads/chromedriver.exe")
url = "https://www.naukri.com"
driver.get(url)
driver.maximize_window()
time.sleep(3)
actions = ActionChains(driver)
windows_before  = driver.current_window_handle

In [3]:
#find recruiter menu
hover_menu = driver.find_elements_by_class_name("mTxt")[1]
actions.move_to_element(hover_menu).perform()
#find "browse all recruiter" menu and click
click_menu = driver.find_element_by_xpath("//a[@title='Browse All Recruiters']")
click_menu.click()
WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
windows_after = driver.window_handles
#switch to new window
new_window = [x for x in windows_after if x != windows_before][0]
driver.switch_to.window(new_window)
time.sleep(10)
#find query element
query = driver.find_element_by_name("qp")
query.clear()
query.send_keys("Data science")
#submit button
driver.find_element_by_id("qsbFormBtn").click()


In [23]:

#initializing info lists
names = []
designations = []
companies = []
skills = []
locations = []
#looping through individual divs
nxt = True
i = 1
while(nxt):
    #finding list of individual divs
    divs = driver.find_elements_by_class_name("recSec")
    for div in divs:
        try:
            names.append(div.find_element_by_xpath(".//span[@class='fl ellipsis']").get_attribute("innerHTML"))
        except Exception as e:
            names.append("--")
        try:
            designations.append(div.find_element_by_xpath(".//span[@class='ellipsis clr']").get_attribute("innerHTML"))
        except:
            designations.append("--")
        try:
            companies.append(div.find_elements_by_xpath(".//a[@class='ellipsis']")[1].text)
        except:
            companies.append("--")
        try:
            skills.append(div.find_element_by_xpath(".//div[@class='hireSec highlightable']").text)
        except:
            skills.append("--")
            
        try:
            locations.append(div.find_element_by_xpath(".//small[@class='ellipsis']").get_attribute("innerHTML"))
        except:
            locations.append("--")
    #clicking the next button
    try:
        buttons = driver.find_elements_by_class_name("grayBtn")
        if len(buttons) == 1:
            if "next" in buttons[0].text.lower():
#                 mal_url = buttons[0].get_attribute("onclick").strip("getpaginationUrl(');").\
#                 split(",")[0].strip("'")
#                 driver.get(mal_url)
                buttons[0].click()
            else:
                nxt = False
        else:
            for button in buttons:
                if "next" in button.text.lower():
                    button.click()
#                     mal_url = button.get_attribute("onclick").strip("getpaginationUrl(');").\
#                 split(",")[0].strip("'")
#                     driver.get(mal_url)
    except Exception as e:
        print(str(e))
        print("hault")
        nxt = False
    time.sleep(10)
        
#creating Dataframe
recruiters = pd.DataFrame({"Name" :names, "Designation" :designations, "Company" :companies, 
                           "Skills" :skills, "Location" :locations })
recruiters

,Name,Designation,Company,Skills,Location
0,Aakash Harit,HR Manager,Data Science Network,"Classic ASP Developer, Internet Marketing Prof...",Delhi
1,shravan Kumar Gaddam,Company Recruiter,Shore Infotech India Pvt. Ltd,".Net, Java, Data Science, Linux Administration...",Hyderabad / Secunderabad
2,MARSIAN Technologies LLP,Company HR,MARSIAN Technologies LLP,"Data Science, Artificial Intelligence, Machine...",Pune
3,Anik Agrawal,Company Recruiter,Enerlytics Software Solutions Pvt Ltd,"Mean Stack, javascript, angularjs, mongodb, We...",Ahmedabad
4,subhas patel,Founder CEO,LibraryXProject,"Hadoop, Spark, Digital Strategy, Data Architec...",UK - (london)
...,...,...,...,...,...
5814,Vikas Tiwari,Tester,Accenture,"Epicor Pos, C++, Sql, Plsql, Sap, Pos",Mumbai
5815,Faraaz Kharwa,Senior Manager,Netscribes (India) Pvt Ltd,Not Specified,Kolkata
5816,Ravi Manda,Sr. Talent Acquisition Recruiter,New York Global Consultants Pvt. Ltd.,Not Specified,Hyderabad / Secunderabad
5817,Ashish Dave,Sales Head-APAC,Cloudiway,Not Specified,Delhi


In [34]:
url = "https://www.theguardian.com/news/datablog/2012/aug/09/"\
"best-selling-books-all-time-fifty-shades-grey-compare"
#getting the response from url
page = requests.get(url)
soup = BeautifulSoup(page.content)
#finding the table
table = soup.find("table")
#finding all rows
trs = table.find_all("tr")
ranks = []
names = []
authors = []
volumes = []
publishers = []
genres = []
# looping through trs
for tr in trs[2:]:
    tds = tr.find_all("td")
    ranks.append(tds[0].text)
    names.append(tds[1].text)
    authors.append(tds[2].text)
    volumes.append(tds[3].text)
    publishers.append(tds[4].text)
    genres.append(tds[5].text)
    
#creating dataframe
novels = pd.DataFrame({"Book Name" :names, "Author" :authors, "Volumes Sold" :volumes,
                      "Publisher" :publishers, "Genre" :genres}, index=ranks)
novels

,Book Name,Author,Volumes Sold,Publisher,Genre
1,"Da Vinci Code,The","Brown, Dan","5,094,805",Transworld,"Crime, Thriller & Adventure"
2,Harry Potter and the Deathly Hallows,"Rowling, J.K.","4,475,152",Bloomsbury,Children's Fiction
3,Harry Potter and the Philosopher's Stone,"Rowling, J.K.","4,200,654",Bloomsbury,Children's Fiction
4,Harry Potter and the Order of the Phoenix,"Rowling, J.K.","4,179,479",Bloomsbury,Children's Fiction
5,Fifty Shades of Grey,"James, E. L.","3,758,936",Random House,Romance & Sagas
...,...,...,...,...,...
96,"Ghost,The","Harris, Robert","807,311",Random House,General & Literary Fiction
97,Happy Days with the Naked Chef,"Oliver, Jamie","794,201",Penguin,Food & Drink: General
98,"Hunger Games,The:Hunger Games Trilogy","Collins, Suzanne","792,187",Scholastic Ltd.,Young Adult Fiction
99,"Lost Boy,The:A Foster Child's Search for the ...","Pelzer, Dave","791,507",Orion,Biography: General


In [46]:
url = "https://www.imdb.com/list/ls095964455"
#getting response
page = requests.get(url)
soup = BeautifulSoup(page.content)
#finding info divs
divs = soup.find_all("div",{"class":["lister-item","mode-detail"]})
names = []
spans = []
genres = []
run_times = []
ratings = []
votes = []
#looping through individual div
for div in divs:
    names.append(div.find("h3").a.text)
    spans.append(div.find("h3").find("span", {"class":["lister-item-year"]}).text.strip("()"))
    genres.append(div.find("span", {"class":["genre"]}).text.strip())
    run_times.append(div.find("span", {"class":["runtime"]}).text.strip())
    ratings.append(div.find("span", {"class":["ipl-rating-star__rating"]}).text.strip())
    votes.append(div.find("span", {"name":["nv"]}).text.strip())

#creating dataframe
shows = pd.DataFrame({"Name" :names, "Year Span" :spans, "Genre" :genres, 
                     "Run Time" :run_times, "Rating" :ratings, "Votes" :votes})
shows

,Name,Year Span,Genre,Run Time,Rating,Votes
0,Game of Thrones,2011–2019,"Action, Adventure, Drama",57 min,9.3,"1,821,460"
1,Stranger Things,2016–,"Drama, Fantasy, Horror",51 min,8.7,"862,599"
2,The Walking Dead,2010–2022,"Drama, Horror, Thriller",44 min,8.2,"873,920"
3,13 Reasons Why,2017–2020,"Drama, Mystery, Thriller",60 min,7.6,"262,485"
4,The 100,2014–2020,"Drama, Mystery, Sci-Fi",43 min,7.6,"223,787"
...,...,...,...,...,...,...
95,Reign,2013–2017,"Drama, Fantasy",42 min,7.5,"44,545"
96,A Series of Unfortunate Events,2017–2019,"Adventure, Comedy, Drama",50 min,7.8,"55,029"
97,Criminal Minds,2005–2020,"Crime, Drama, Mystery",42 min,8,"167,430"
98,Scream: The TV Series,2015–2019,"Comedy, Crime, Drama",45 min,7.1,"34,858"


In [67]:
#starting web driver
driver = webdriver.Chrome(executable_path = r"C:/Users/DEVASREE/Downloads/chromedriver.exe")
url = "https://archive.ics.uci.edu"
driver.get(url)
driver.maximize_window()
time.sleep(3)

In [68]:
#find navigation items
spans = driver.find_elements_by_class_name("whitetext")
#looping through nav menus to click "View ALL datasets"
for span in spans:
    if "view all data sets" in span.text.lower():
        span.find_element_by_tag_name("a").click()
        break     

In [114]:
#finding the table which contains the dataset info
table = driver.find_element_by_xpath("//table[@border='1'][@cellpadding='5']")
#finding the rows of that table
trs = table.find_elements_by_tag_name("tr")
#initializing the info. lists
names = []
data_types = []
tasks = []
at_types = []
no_instances = []
no_attrs = []
years = []
#looping through trs starts from 1 to avoid heading. looping alternate rows to avoid inner <tr>-
# -on the first td
for tr in trs[1:-1:2]:
    tds = tr.find_elements_by_tag_name("td")
    names.append(tds[0].find_element_by_tag_name("tr").text.strip())
    data_types.append(tds[3].find_element_by_class_name("normal").text)
    tasks.append(tds[4].text)
    at_types.append(tds[5].text)
    no_instances.append(tds[6].text)
    no_attrs.append(tds[7].find_element_by_class_name("normal").text)
    years.append(tds[8].text)

#creating dataframe
datasets = pd.DataFrame({"Name" :names, "Data Type" :data_types, "Task" :tasks,
                        "Attribute Type" :at_types, "No. of Instances" :no_instances,
                        "No. of Attributes" :no_attrs, "Year" :years})
datasets

,Name,Data Type,Task,Attribute Type,No. of Instances,No. of Attributes,Year
0,Abalone,Multivariate,Classification,"Categorical, Integer, Real",4177,8,1995
1,Adult,Multivariate,Classification,"Categorical, Integer",48842,14,1996
2,Annealing,Multivariate,Classification,"Categorical, Integer, Real",798,38,
3,Anonymous Microsoft Web Data,,Recommender-Systems,Categorical,37711,294,1998
4,Arrhythmia,Multivariate,Classification,"Categorical, Integer, Real",452,279,1998
...,...,...,...,...,...,...,...
583,in-vehicle coupon recommendation,Multivariate,Classification,,12684,23,2020
584,Gait Classification,Multivariate,Classification,Real,48,321,2020
585,Wikipedia Math Essentials,Time-Series,Regression,Real,731,1068,2021
586,Wikipedia Math Essentials,Time-Series,Regression,Real,731,1068,2021
